In [21]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#For oversampling
from collections import Counter
from imblearn.over_sampling import ADASYN
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier

In [38]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\PCDataset.csv")
#Removing irrelevant columns
#df = df.drop(['Week', 'PlayerName' , 'CalendarDate'], axis = 1)
features = df.drop(['InjuredNextTrainingSession'], axis = 1)
target = df['InjuredNextTrainingSession']

In [12]:
#Standardising the columns - mean 0, variance 1
features_transform = StandardScaler().fit_transform(features)
#Creating PCA object with four principal components
pca = PCA(n_components = 4)
pcomponents = pca.fit(features_transform)
#Outputting the PCA values
print("Explained Variance: %s" % pcomponents.explained_variance_ratio_)
print(pcomponents.components_)

Explained Variance: [0.43660067 0.14149759 0.1107006  0.06874266]
[[ 0.22873921  0.16611251  0.22702291  0.19124376  0.16999673  0.2576973
   0.21465697  0.19911335 -0.04522897  0.26562929  0.18289685  0.25332393
   0.19436357  0.24579131  0.20360249  0.19242583  0.27110492  0.24044532
   0.23700206 -0.04350971  0.27769617  0.19377757]
 [-0.1618458   0.2100621   0.2400642   0.21224491  0.29164135 -0.10603449
  -0.26763648 -0.12175824 -0.35166556 -0.18831076 -0.09026004 -0.11813736
   0.16710145  0.23771659  0.22872979  0.29973408 -0.08419058 -0.2256052
  -0.11662231 -0.36616533 -0.14755963 -0.09480788]
 [-0.20085106  0.37367102 -0.00240471 -0.27522011 -0.25657178  0.02461785
  -0.23992365  0.15703999 -0.1134785   0.02626824  0.3406409  -0.15863532
   0.37393124  0.00496651 -0.27258907 -0.22536191  0.08313592 -0.20344952
   0.11664602 -0.10379442  0.06708432  0.31420203]
 [ 0.35956085 -0.02436398 -0.14436122 -0.07701928 -0.16228449 -0.10064843
   0.20497227 -0.04969645 -0.52298314 -0.03

In [3]:
#Split dataset into 70% training set, 30% test set; random state of 1 ensures
#same distribution every time
#x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 1)
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(target)
print(imbal_proportion)
#Oversampling the training set
oversample = ADASYN()
features_test, target_test = oversample.fit_resample(features, target)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(target_test)
print(bal_proportion)

Counter({0: 3805, 1: 38})
Counter({0: 3805, 1: 3798})


In [4]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    acc = (TP + TN) / (real_values_one + real_values_zero)
    f1 = (2*TP) / (2*TP + FP + FN)
    pre = TP/(TP+FP)
    
    return sensitivity*100, specificity*100, acc*100, f1*100, pre*100, TP, FP, TN, FN

In [55]:
#Function to return prediction outcomes
def preditorini(test_pred, test_real):
    count = Counter(test_real)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    test_pred = list(map(int, test_pred))
    test_real = list(map(int, test_real))
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for i in range(len(test_pred)):
        if test_pred[i] == 1 and test_real[i] == 1:
            TP += 1
        if test_real[i] == 0 and test_pred[i] != test_real[i]:
            FP += 1
        if test_pred[i] == test_real[i] == 0:
            TN += 1
        if test_real[i] == 1 and test_pred[i] != test_real[i]:
            FN += 1
            
    return real_values_zero, real_values_one, TP, TN, FP, FN

In [103]:
train_set = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\CompleteDatasetApproach2.csv")
#Removing irrelevant columns
train_set = train_set.drop(['Week', 'PlayerName'], axis = 1)
train_features = train_set.drop(['InjuredNextWeek'], axis = 1)
train_target = train_set['InjuredNextWeek']

test_set = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\PredB.csv")
test_set = test_set.drop(['Week', 'PlayerName'], axis = 1)
test_features = test_set.drop(['InjuredNextWeek'], axis = 1)
reals = test_set['InjuredNextWeek']

model = RandomForestClassifier(criterion = 'entropy', max_depth = 10)
features_oversample, target_oversample = oversample.fit_resample(train_features, train_target)
predictor = model.fit(features_oversample, target_oversample)
machines = predictor.predict(test_features)
preditorini(machines, reals)

(441, 8, 4, 441, 0, 4)

In [80]:
#Finding best hyperparameters for gini impurity Decision Tree
#parameters = {'max_depth':range(3,41)}
#DTgini = GridSearchCV(DecisionTreeClassifier(criterion = 'gini'), parameters, n_jobs = 4)
#DTgini = DTgini.fit(X = x_train, y = y_train)
#print(DTgini.best_score_, DTgini.best_params_)

#Finding best hyperparameters for entropy Decision Tree
#DTent = GridSearchCV(DecisionTreeClassifier(criterion = 'entropy'), parameters, n_jobs = 4)
#DTent = DTent.fit(X = x_train, y = y_train)
#print(DTent.best_score_, DTent.best_params_)


In [115]:
from sklearn.tree import export_text
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\PCDatasetApproach2.csv")
#Removing irrelevant columns
#df = df.drop(['Week', 'PlayerName'], axis = 1)
features = df.drop(['InjuredNextWeek'], axis = 1)
target = df['InjuredNextWeek']

myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)
features_oversample, target_oversample = oversample.fit_resample(features, target)
DTC = myDT.fit(features_oversample, target_oversample)
r = export_text(DTC, feature_names=list(features.columns))
print(r)

|--- EWMA_AccelLoadDensity <= 0.42
|   |--- EWMA_Distance <= 3877.73
|   |   |--- EWMA_AccelLoadDensity <= 0.32
|   |   |   |--- EWMA_AccelLoadDensity <= 0.29
|   |   |   |   |--- EWMA_AccelLoadTotal <= 1145.69
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- EWMA_AccelLoadTotal >  1145.69
|   |   |   |   |   |--- AccelMetres <= 79.46
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- AccelMetres >  79.46
|   |   |   |   |   |   |--- EWMA_AccelLoadTotal <= 1267.02
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- EWMA_AccelLoadTotal >  1267.02
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |--- EWMA_AccelLoadDensity >  0.29
|   |   |   |   |--- EWMA_AccelLoadTotal <= 899.71
|   |   |   |   |   |--- EWMA_Distance <= 2217.61
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- EWMA_Distance >  2217.61
|   |   |   |   |   |   |--- EWMA_AccelMetres <= 70.59
|   |   |   |   |   |   |   |--- HSR <= 36.82
|   |   |   |   |   |   |   |   

In [51]:
r[5]

'A'

In [32]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

KeyboardInterrupt: 

In [8]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

TP: 341279 (44.887412863343414%), FP: 69274 (9.111403393397342%)
TN: 311226 (40.934631066684204%), FN: 38521 (5.066552676575037%)
Sensitivity has mean 89.85755660874143 with variance 1.4608766261226165
Specificity has mean 81.79395532194481 with variance 0.6237676754944136
ACC has mean 85.82204393002763 with variance 0.4093525768407693
Precision has mean 83.130280437287 with variance 0.3518725073178953
F1-Score has mean 86.35841928029521 with variance 0.44364899181573675


In [9]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Logistic Regression classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

...

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

TP: 247239 (32.51861107457582%), FP: 160628 (21.126923582796266%)
TN: 219872 (28.91911087728528%), FN: 132561 (17.435354465342627%)
Sensitivity has mean 65.09715639810427 with variance 0.3932655861389865
Specificity has mean 57.7850197109067 with variance 0.16826991250533102
ACC has mean 61.43772195186111 with variance 0.14878796623351861
Precision has mean 60.617067480080934 with variance 0.11346942880426143
F1-Score has mean 62.776565430552665 with variance 0.19434138077980623


C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

TP: 303930 (39.97500986452716%), FP: 219773 (28.906089701433643%)
TN: 160727 (21.139944758647903%), FN: 75870 (9.978955675391292%)
Sensitivity has mean 80.02369668246445 with variance 0.22453684639319932
Specificity has mean 42.24099868593955 with variance 0.05836231115777183
ACC has mean 61.11495462317505 with variance 0.08087538702342886
Precision has mean 58.03446804798032 with variance 0.03637946567206813
F1-Score has mean 67.27770534527923 with variance 0.08100180729086903


In [11]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

TP: 307994 (40.50953570958832%), FP: 233260 (30.679994738918847%)
TN: 147240 (19.3660397211627%), FN: 71806 (9.444429830330133%)
Sensitivity has mean 81.09373354397052 with variance 0.476967589992904
Specificity has mean 38.69645203679369 with variance 0.13901067307177611
ACC has mean 59.875575430751006 with variance 0.10224689441469183
Precision has mean 56.90323422160015 with variance 0.040816682634463704
F1-Score has mean 66.87756143609833 with variance 0.12277398089768492


In [12]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

TP: 350435 (46.09167433907668%), FP: 43214 (5.6838090227541755%)
TN: 337286 (44.36222543732737%), FN: 29365 (3.862291200841773%)
Sensitivity has mean 92.26829910479199 with variance 0.27073331464330475
Specificity has mean 88.64283837056504 with variance 0.14130407980370227
ACC has mean 90.45389977640407 with variance 0.08765341439444027
Precision has mean 89.02332058541754 with variance 0.10207333484210784
F1-Score has mean 90.61576967573282 with variance 0.0916102099711738


In [20]:
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))

TP: 331096 (43.536620644312954%), FP: 49533 (6.513214990138067%)
TN: 330967 (43.51965811965812%), FN: 48904 (6.430506245890861%)
Sensitivity has mean 87.1305263157895 with variance 0.390926869806094
Specificity has mean 86.98212877792379 with variance 0.16329651316391647
ACC has mean 87.05627876397108 with variance 0.1358829725936381
Precision has mean 86.98748772405096 with variance 0.1286330429722312
F1-Score has mean 87.05772406459566 with variance 0.1529606654826353


In [ ]:
# Baseline 4: Unsampled
i = 1
sensitivities = []
specificities = []
ACCs = []
F1s = []
PREs = []
TPs = []
FPs = []
TNs = []
FNs = []

myRF = RandomForestClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    ACCs.append(performance[2])
    F1s.append(performance[3])
    PREs.append(performance[4])
    TPs.append(performance[5])
    FPs.append(performance[6])
    TNs.append(performance[7])
    FNs.append(performance[8])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specifici40ties))+" with variance "+str(np.var(specificities)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Precision has mean "+str(np.mean(PREs))+" with variance "+str(np.var(PREs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))